In [22]:
import os
import numpy as np
from numpy import array, ma, genfromtxt
import pandas as pd

In [23]:
# The CANlab/WAVi Pain Study is Pre-Formatted to resemble BIDS neuroimaging formatting
# If your study does not abide to the following structure, please revisit previous scripts
# .../StudyRepo
# -------------> /raw
# -------------------> /*.art
# -------------------> /*.eeg
# -------------------> /*.evt
# -------------> /task1
# -------------------> /group1
# ------------------------> /sub1_ftype1.csv
# ------------------------> /sub1_ftype2.csv
# ------------------------> /...
# ------------------------> /subn_ftypen.csv
# -------------------> /...
# -------------------> /groupn
# -------------> /...
# -------------> /taskn

In [24]:
# Before running this script, you should already have 
# converted your raw data into CSV formats as shown above

In [25]:
print("Please enter the full path of the study repository:")
directory = input()

while(not os.path.exists(directory)):
    print("\n")
    print("The directory you supplied could not be found.")
    print("Please enter the full path of the study directory containing EEG, ART, EVT files:")
    directory = input()
    
os.chdir(directory)

Please enter the full path of the study repository:
/home/clayton/science/CANlab/WAViMedEEG/CANlabStudy


In [26]:
print("Which task would you like to load? \n p300? \n flanker? \n chronic? \n rest?")
task = ""
while(task == ""):
    task = input() 
os.chdir(task)

Which task would you like to load? 
 p300? 
 flanker? 
 chronic? 
 rest?
p300


In [27]:
if(os.getcwd() is not (directory+"/"+task)):
    os.chdir(directory+"/"+task)

class taskInfo():
    pain_fnames = os.listdir("1")
    ctrl_fnames = os.listdir("2")
    pain_subs = []
    ctrl_subs = []
    for fname in pain_fnames:
        if(fname[:3] not in pain_subs):
            pain_subs.append(fname[:3])
    for fname in ctrl_fnames:
        if(fname[:3] not in ctrl_subs):
            ctrl_subs.append(fname[:3])
    
def load_csv(sub):
    eeg = genfromtxt(sub+"_eeg.csv", delimiter=",")
    art = genfromtxt(sub+"_art.csv", delimiter=",")
    return(eeg, art)

def apply_art_mask(data, artifact):
    mx = ma.masked_array(data, mask=artifact)
    mx = ma.filled(mx.astype(float), np.nan)
    return(mx)

#provide length as num of cycles, 250 Hz freq
def find_contigs(min_length):
    return(null)
    
def load_p300():
    p300_pain = []
    p300_ctrl = []
    info = taskInfo()
    os.chdir("1")
    for sub in info.pain_subs:
        eeg, art = load_csv(sub)
        p300_pain.append(apply_art_mask(eeg, art))
    os.chdir("..")
    os.chdir("2")
    for sub in info.ctrl_subs:
        eeg, art = load_csv(sub)
        p300_ctrl.append(apply_art_mask(eeg, art))
    os.chdir("..")
    return(p300_pain, p300_ctrl)

def load_flanker():
    flanker_pain = []
    flanker_ctrl = []
    info = taskInfo()
    os.chdir("1")
    for sub in info.pain_subs:
        eeg, art = load_csv(sub)
        flanker_pain.append(apply_art_mask(eeg, art))
    os.chdir("..")
    os.chdir("2")
    for sub in info.ctrl_subs:
        eeg, art = load_csv(sub)
        flanker_ctrl.append(apply_art_mask(eeg, art))
    os.chdir("..")
    return(flanker_pain, flanker_ctrl)
    
def load_chronic():
    p300_pain = []
    p300_ctrl = []
    info = taskInfo()
    os.chdir("1")
    for sub in info.pain_subs:
        eeg, art = load_csv(sub)
        chronic_pain.append(apply_art_mask(eeg, art))
    os.chdir("..")
    os.chdir("2")
    for sub in info.ctrl_subs:
        eeg, art = load_csv(sub)
        chronic_ctrl.append(apply_art_mask(eeg, art))
    os.chdir("..")
    return(chronic_pain, chronic_ctrl)
    
def load_rest():
    rest_pain = []
    rest_ctrl = []
    info = taskInfo()
    os.chdir("1")
    for sub in info.pain_subs:
        eeg, art = load_csv(sub)
        rest_pain.append(apply_art_mask(eeg, art))
    os.chdir("..")
    os.chdir("2")
    for sub in info.ctrl_subs:
        eeg, art = load_csv(sub)
        rest_ctrl.append(apply_art_mask(eeg, art))
    os.chdir("..")
    return(rest_pain, rest_ctrl)
    
pain, control = load_p300()
# You will now have 2 stacks of full-length masked eeg data
# One for patients, one for controls

In [28]:
print("Number of patients: " + str(len(pain)))
print("Number of controls: " + str(len(control)))
print("Total N: " + str(len(pain)+len(control)))
print(" \nThat's not a lot! Let's get more by arranging the data into contigs of some desired length.")

Number of patients: 9
Number of controls: 5
Total N: 14
 
That's not a lot! Let's get more by arranging the data into contigs of some desired length.


In [29]:
def sec_to_cyc(seconds):
    return(seconds * 250)
        
def generate_contigs(run, length):
    i = 0
    contigs = []
    startindexes = []
    while i < run.shape[0]-length:
        stk = run[i:(i+length),:]
        if not np.any(np.isnan(stk)):
            contigs.append(stk)
            startindexes.append(i)
            i += length
        else:
            i+=1
    return contigs, startindexes

In [32]:
def get_contigs_from_trials(trials):
    all_contigs = []
    print("How many seconds long should each contig be?")
    cycles = sec_to_cyc(int(input()))
    for run in trials:
        all_contigs.append(generate_contigs(run, cycles))
    return(all_contigs)

In [42]:
pain_contigs = get_contigs_from_trials(pain)

How many seconds long should each contig be?
3


In [41]:
pain_contigs[1]

([array([[  1.,  -9.,  -4., ...,   1.,  -3.,  -6.],
         [  4.,  -7.,  -1., ...,   3.,   0.,  -2.],
         [  6.,  -5.,   0., ...,   4.,   3.,   1.],
         ...,
         [  0.,   4.,   0., ...,   0.,  -3., -17.],
         [  0.,   5.,   0., ...,   0.,  -5., -18.],
         [  1.,   6.,   0., ...,   1.,  -6., -19.]])], [48992])

In [ ]:
# def load_per_task(task):
#     switcher = {
#         p300: load_p300,
#         flanker: load_flanker,
#         chronic: load_chronic,
#         rest: load_rest
#     }
#     func = switcher.get(argument, lambda: "Invalid task")